
L'obiettivo del progetto è apprendere mediante una rete neurale la trasformazione da punti del piano espressi in coordinate polari ad una rappresentazione basata su di una griglia discreta di dimensione 10x10, dove la cella della griglia ha valore 1 se contiene il punto, e 0 altrimenti.

Il dataset supervisionato è fornito in questo notebook nella forma di una generatore. Il generatore deve essere considerato come una "scatola nera" il cui comportamento deve essere appreso. 

Dovete progettare una rete neurale in grado di raggiungere una accuratezza del 95%. Questa è una condizione necessaria per superare l'esame, ma l'accuratezza non influisce in altro modo sulla valutazione.  

I modelli che raggiungono l'accuratezza attesa saranno invece valutati in modo inversamente proporzionale al numero dei loro parametri: **più il modello è piccolo, meglio è.**


**Attenzione**: Qualunque soluzione che tragga vantaggio, diretto o indiretto, da meta-conoscenza relativa al generatore sarà automaticamente bocciato.


Veniamo al generatore. Questo restituisce delle triple della forma
((theta,rho),out) dove (theta,rho) sono le coordinate polari di un punto nel primo quadrante del piano, e out è una mappa 10x10 con "1" in correspondenza alla cella che contiene il punto, e "0" altrimenti.

Settando  flat=True, la mappa 10x10 viene appiattita ad un vettore di dimensione 100. Potete utilizzare questa variante, se preferite. Nessuna altra modifica del generatore è ammessa. 

Creiamo una istanza del generatore con una griglia di dimensione 3x4

... e osserviamo qualche esempio

In [1]:
import keras
from keras import layers
from keras import activations
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

def polar_generator(batchsize,grid=(10,10),noise=.002,flat=False):
  while True:
    x = np.random.rand(batchsize)
    y = np.random.rand(batchsize)
    out = np.zeros((batchsize,grid[0],grid[1]))
    xc = (x*grid[0]).astype(int)
    yc = (y*grid[1]).astype(int)
    for b in range(batchsize):
      out[b,xc[b],yc[b]] = 1
    #compute rho and theta and add some noise
    rho = np.sqrt(x**2+y**2) + np.random.normal(scale=noise)
    theta = np.arctan(y/np.maximum(x,.00001)) + np.random.normal(scale=noise)
    if flat:
      out = np.reshape(out,(batchsize,grid[0]*grid[1]))
    yield ((theta,rho),out)

n_train = 4000000
n_test = 20000
batch_size = 2048

g1,g2 = 10,10
gen = polar_generator(n_train+n_test,grid=(g1,g2),noise=0.002,flat=True)
# (theta,rho),y = next(gen)
(theta,rho),y = next(gen)

x=np.array([i for i in zip(theta,rho)])

# def xy(i):
#   (theta,rho),y = i
#   x=np.array([i for i in zip(theta,rho)])
#   return (x,y)

# gcd = np.gcd(n_train, n_test)
# gen = pol.ipynbar_generator(gcd,grid=(size,size),noise=0.02)

# x_train, y_train = xy(next(gen))
# for i in range(int(n_train/gcd)-1):
#   x,y = xy(next(gen))
#   x_train = np.concatenate((x, x_train), axis=0)
#   y_train = np.concatenate((y, y_train), axis=0)

# x_test, y_test = xy(next(gen))
# for i in range(int(n_train/gcd)-1):
#   x,y = xy(next(gen))
#   x_test = np.concatenate((x, x_test), axis=0)
#   y_test = np.concatenate((y, y_test), axis=0)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=n_test/(n_train+n_test), shuffle=True, random_state=1)


def discretized_accuracy(true_maps: tf.Tensor, my_maps: tf.Tensor) -> float:
  equals = tf.equal(tf.argmax(true_maps, axis=1), tf.argmax(my_maps, axis=1))
  return tf.cast(tf.math.count_nonzero(equals), tf.float64) / tf.cast(len(true_maps), tf.float64)

2023-01-11 17:46:20.764659: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [37]:
network = keras.Sequential([
layers.Flatten(),
# layers.Dense(20, activation=keras.activations.relu),
# layers.Dense(20, activation=keras.activations.tanh),
# layers.Dropout(0.1), # Spegni il 10% dei neuroni a caso
layers.Dense(4, activation=keras.activations.softsign),
layers.Dense(6, activation=keras.activations.swish),
layers.Dense(6, activation=keras.activations.tanh),
# layers.BatchNormalization(),
# layers.Dropout(0.5), # Spegni il 10% dei neuroni a caso
layers.Dense(6, activation=keras.activations.relu),
# layers.Dropout(0.5), # Spegni il 10% dei neuroni a caso
# layers.LayerNormalization(),
# layers.BatchNormalization(),
layers.Dense(100, activation=activations.softmax)
])

# Prima di poter usare il modello dobbiamo dire a Keras la dimensione dei nostri input
# "None" vuol dire che il numero è ignoto/può cambiare (perché quante immagini alla volta g# può cambiare)
network.build((None, 2))
network.summary()

network.compile(
optimizer=keras.optimizers.Adam(learning_rate=1e-3), # Impostiamo SGD come ottimizzatore
loss=keras.losses.CategoricalCrossentropy(),
# loss=customLoss,
metrics=['accuracy', discretized_accuracy]
)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 2)                 0         
                                                                 
 dense_10 (Dense)            (None, 4)                 12        
                                                                 
 dense_11 (Dense)            (None, 6)                 30        
                                                                 
 dense_12 (Dense)            (None, 6)                 42        
                                                                 
 dense_13 (Dense)            (None, 6)                 42        
                                                                 
 dense_14 (Dense)            (None, 100)               700       
                                                                 
Total params: 826
Trainable params: 826
Non-trainable 

In [3]:
network = keras.models.load_model("mymodel")

ValueError: ignored

In [38]:
history = network.fit(
x=x_train,
y=y_train,
epochs=15, # Addestriamo per 100 epoche
batch_size=batch_size, # Usiamo una batch size di 128
validation_data=(x_test, y_test),
callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
)

Epoch 1/15
1954/1954 [==============================] - 14s 7ms/step - loss: 3.1988 - accuracy: 0.1112 - discretized_accuracy: 0.1112 - val_loss: 2.3723 - val_accuracy: 0.2682 - val_discretized_accuracy: 0.2679
Epoch 2/15
1954/1954 [==============================] - 13s 7ms/step - loss: 1.5706 - accuracy: 0.5845 - discretized_accuracy: 0.5845 - val_loss: 1.0518 - val_accuracy: 0.7510 - val_discretized_accuracy: 0.7506
Epoch 3/15
1954/1954 [==============================] - 16s 8ms/step - loss: 0.8147 - accuracy: 0.8176 - discretized_accuracy: 0.8176 - val_loss: 0.6451 - val_accuracy: 0.8601 - val_discretized_accuracy: 0.8601
Epoch 4/15
1954/1954 [==============================] - 15s 8ms/step - loss: 0.5461 - accuracy: 0.8815 - discretized_accuracy: 0.8816 - val_loss: 0.4653 - val_accuracy: 0.8994 - val_discretized_accuracy: 0.8995
Epoch 5/15
1954/1954 [==============================] - 14s 7ms/step - loss: 0.4132 - accuracy: 0.9120 - discretized_accuracy: 0.9120 - val_loss: 0.3682 - v

In [33]:
score, _, acc  = network.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Accuracy: {:.1f}%'.format(acc*100))

10/10 [==============================] - 0s 9ms/step - loss: 0.0996 - accuracy: 0.9783 - discretized_accuracy: 0.9782
Test score: 0.09960493445396423
Accuracy: 97.8%


In [34]:
network.save("mymodel_2048")
network.summary()

INFO:tensorflow:Assets written to: mymodel_2048/assets
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 2)                 0         
                                                                 
 dense_5 (Dense)             (None, 4)                 12        
                                                                 
 dense_6 (Dense)             (None, 8)                 40        
                                                                 
 dense_7 (Dense)             (None, 8)                 72        
                                                                 
 dense_8 (Dense)             (None, 6)                 54        
                                                                 
 dense_9 (Dense)             (None, 100)               700       
                                                                 

In [36]:
gen = polar_generator(20000,grid=(g1,g2),noise=0.002,flat=True)

accs = 0.0
iters = 100
for x in range(iters):
  (theta,rho),y = next(gen)
  x=np.array([i for i in zip(theta,rho)])

  score, _, acc = network.evaluate(x, y, batch_size=batch_size)
  accs += acc

print('Accuracy: {:.1f}%'.format(accs/iters*100))

10/10 [==============================] - 0s 5ms/step - loss: 0.1178 - accuracy: 0.9592 - discretized_accuracy: 0.9592
Accuracy: 95.0%


Utile esercizio: aggiungete rumore al generatore e verificate l'effetto sulla
"ground truth".

# Cosa consegnare

Ai fini del progetto dovete lavorare con la **griglia di default di dimensione 10x10, e con il rumore di default .002**

il generatore deve essere trattato come una scatola nera: non modificatelo e non sfruttate la sua semantica, che si suppone ignota. Potete lavorare in modlaità "flat", se preferite.

Dovete:

1.   definire una funzione per il calcolo della accuratezza (potete prendere ispirazione dal cocice della cella precedente) 
2.   definire una rete neurale che prende in input theta e rho e restituisce out
3.  misurare l'accuratezza della rete, che deve essere maggiore o uguale del 95%; l'accuratezza deve essere misurata su almeno 20000 dati
4. perfezionare il modello cercando di diminuire il più possibile il numero dei parametri mantenendo una accuratezza superiore al 95%. Solo la vostra rete migliore deve essere consegnata.

Dovete consegnare un UNICO notebook eseguibile su colab, che contenga il codice della rete, il suo sommario con il numero dei parametri, la storia di training, il codice per il calcolo della accuratezza e la sua valutazione sulla vostra rete.

**N.B.** L'accuratezza deve essere superiore o uguale a 95%, ma non influisce in altro modo sulla valutazione. Il vostro punteggio dipenderà unicamente dal numero dei parametri: più è piccolo e più la vostra vaalutazione sarà elevata.  

#Buon lavoro!



